In [1]:
import pandas as pd
import numpy as np
import google_analytics_reporting_api as ga
import json as json
import pandas as pd
import numpy as np
import datetime
import googlemaps
import glob
# google_analytics_reporting_api functions writen by Jay in the path '/home/jian/Projects/Saatva/GA/google_analytics_reporting_api.py'

In [14]:
Zip_Deduplated=pd.read_excel("/home/jian/Projects/Saatva/Otherinput/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
Zip_Deduplated=Zip_Deduplated.iloc[:,[0,2,6]]
Zip_Deduplated.columns=['zipcode','dma','ST']
Zip_Deduplated=Zip_Deduplated.drop_duplicates(['zipcode'])
Zip_Deduplated.reset_index(inplace=True)
del Zip_Deduplated['index']
Zip_Deduplated=Zip_Deduplated[Zip_Deduplated['ST']!="HI"]
Zip_Deduplated=Zip_Deduplated[Zip_Deduplated['ST']!="AK"]

Saatva=pd.read_excel("/home/jian/Projects/Saatva/Data/Order_data.xlsx",sheetname="Saatva",dtype=str)
Loom_and_Leaf=pd.read_excel("/home/jian/Projects/Saatva/Data/Order_data.xlsx",sheetname="Loom_and_Leaf",dtype=str)
Zenhaven=pd.read_excel("/home/jian/Projects/Saatva/Data/Order_data.xlsx",sheetname="Zenhaven",dtype=str)

In [15]:
Saatva=Saatva.iloc[:,0:7]
Loom_and_Leaf=Loom_and_Leaf.iloc[:,0:7]
Zenhaven=Zenhaven.iloc[:,0:7]


In [16]:
Saatva['date']=Saatva['date'].apply(lambda x:x[0:10])
Saatva['date']=Saatva['date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
Loom_and_Leaf['date']=Loom_and_Leaf['date'].apply(lambda x:x[0:10])
Zenhaven['date']=Zenhaven['date'].apply(lambda x:x[0:10])

In [17]:
Loom_and_Leaf['date_time']=Loom_and_Leaf['date'].apply(lambda x:x[0:10])+" "+Loom_and_Leaf['time'].apply(lambda x:x[0:10])
Loom_and_Leaf['date_time']=Loom_and_Leaf['date_time'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
Loom_and_Leaf['date_time']=Loom_and_Leaf['date_time'].apply(lambda x: x-datetime.timedelta(hours=5))
Loom_and_Leaf['date']=Loom_and_Leaf['date_time'].apply(lambda x:x.date())
Loom_and_Leaf['time']=Loom_and_Leaf['date_time'].apply(lambda x: str(x.time()))
del Loom_and_Leaf['date_time']

In [18]:
Zenhaven['date_time']=Zenhaven['date'].apply(lambda x:x[0:10])+" "+Zenhaven['time'].apply(lambda x:x[0:10])
Zenhaven['date_time']=Zenhaven['date_time'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
Zenhaven['date_time']=Zenhaven['date_time'].apply(lambda x: x-datetime.timedelta(hours=5))
Zenhaven['date']=Zenhaven['date_time'].apply(lambda x:x.date())
Zenhaven['time']=Zenhaven['date_time'].apply(lambda x: str(x.time()))
del Zenhaven['date_time']

In [19]:
Sales_2017=Saatva.append(Loom_and_Leaf)
Sales_2017=Sales_2017.append(Zenhaven)
Sales_2017.reset_index(inplace=True)
del Sales_2017['index']


In [20]:
Sales_2018=pd.read_csv("/home/jian/Projects/Saatva/Data/transactions_for_jubaplus-20180306.csv",dtype=str)
Sales_2018.columns=Sales_2017.columns
Sales_2018=Sales_2018[~pd.isnull(Sales_2018['order_id'])]
Sales_2018.reset_index(inplace=True)
del Sales_2018['index']
Sales_2018['date']=Sales_2018['date'].apply(lambda x: datetime.datetime.strptime(x,"%d-%b-%Y").date())

In [21]:
Sales=Sales_2017.append(Sales_2018)
Sales.reset_index(inplace=True)
del Sales['index']
Sales=Sales[Sales['date']<datetime.datetime.strptime("20180301","%Y%m%d").date()]

In [22]:
# Clean zips in the 50 states of USA
Sales['zipcode']=Sales['zipcode'].astype(str).apply(lambda x:x.split("-")[0])
Sales['zipcode']=Sales['zipcode'].apply(lambda x: np.where(len(x)<5,x.zfill(5),x))
Sales=Sales[Sales['zipcode'].apply(lambda x: len(x))==5]

Sales['product_revenue']=Sales['product_revenue'].astype(float)
Sales['brand']=Sales['brand'].replace("LoomAndLeaf","Loom_and_Leaf")

Sales['zip_num_index']=pd.to_numeric(Sales['zipcode'], errors='coerce')
Sales=Sales[~pd.isnull(Sales['zip_num_index'])]
Sales.reset_index(inplace=True)
del Sales['zip_num_index']
Sales['online/phone?']=Sales['online/phone?'].replace('phone_order','phone')

In [23]:
# Time range: from 2016-11-01 to 2018-02-28
print("from "+str(min(Sales['date']))+" to "+str(max(Sales['date'])))
groupby_func={'product_revenue':['sum','count']}
Sales_by_Day_Long=Sales.groupby(['brand','date'])['product_revenue'].agg(groupby_func)
Sales_by_Day_Long.columns=Sales_by_Day_Long.columns.get_level_values(0)
Sales_by_Day_Long.columns=['sales','transaction']
Sales_by_Day_Long.reset_index(inplace=True)
Sales_by_Day_Long['AOV']=round(Sales_by_Day_Long['sales']/Sales_by_Day_Long['transaction'],2)

# Sales_by_Day_Long=pd.merge(Sales_by_Zip_Long,Zip_Deduplated_All3,on=['zipcode','brand'],how='right')
# Sales_by_Zip_Long is good for Tableau

from 2016-10-31 to 2018-02-28


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


In [24]:
Saatva_by_Day=Sales_by_Day_Long[Sales_by_Day_Long['brand']=="Saatva"]

In [2]:
analytics = ga.initialize_analyticsreporting('/home/jian/Projects/Saatva/GA/GA_api_jubapluscc@gmail.com.json')

def num_func_trans(df):
    if 'ga:sessions' in df.columns:
        df['ga:sessions']=df['ga:sessions'].astype(int)
    if 'ga:users' in df.columns:
        df['ga:users']=df['ga:users'].astype(int)
    if 'ga:pageviews' in df.columns:
        df['ga:pageviews']=df['ga:pageviews'].astype(int)
    if 'ga:bounces' in df.columns:
        df['ga:bounces']=df['ga:bounces'].astype(int)
    if 'ga:bounceRate' in df.columns:
        df['ga:bounceRate']=round(df['ga:bounceRate'].astype(float)/100,4)
    if 'ga:pageviews' in df.columns:
        df['ga:pageviews']=df['ga:pageviews'].astype(int)
    if 'ga:uniquePageviews' in df.columns:
        df['ga:uniquePageviews']=df['ga:uniquePageviews'].astype(int)
    if 'ga:totalEvents' in df.columns:
        df['ga:totalEvents']=df['ga:totalEvents'].astype(int)
    if 'ga:uniqueEvents' in df.columns:
        df['ga:uniqueEvents']=df['ga:uniqueEvents'].astype(int)
    if 'ga:goalCompletionsAll' in df.columns:
        df['ga:goalCompletionsAll']=df['ga:goalCompletionsAll'].astype(int)
    if 'ga:dateHourMinute' in df.columns:
        df['ga:dateHourMinute']=df['ga:dateHourMinute'].apply(lambda x:x[0:4]+"-"+x[4:6]+"-"+x[6:8]+" "+x[8:10]+":"+x[10:12])
    if 'ga:date' in df.columns:
        df['ga:date']=df['ga:date'].apply(lambda x:x[0:4]+"-"+x[4:6]+"-"+x[6:8])
    
    return df

Start_Date='2016-11-01'
End_Date='2018-02-28'

In [35]:
GA_Saatva_Day=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='33605697',start_date="2018-02-01", end_date="2018-02-28",
                                            ga_metrics=['ga:sessions','ga:pageviews','ga:bounceRate','ga:bounces'],
                                            ga_dimensions=['ga:date','ga:country','ga:region'])
GA_Saatva_Day=GA_Saatva_Day[['ga:date','ga:country','ga:region','ga:sessions','ga:pageviews','ga:bounces','ga:bounceRate']]
GA_Saatva_Day.reset_index(inplace=True)
del GA_Saatva_Day['index']

# GA_Saatva_Day=GA_Saatva_Day # 1234 # Exclude non us and 48 states data
GA_Saatva_Day['ga:sessions']=GA_Saatva_Day['ga:sessions'].astype(float)

Change here

In [36]:
GA_Saatva_Day['ga:sessions'].sum()

1167877.0

In [37]:
GA_Saatva_Day_After_ContryUS=GA_Saatva_Day[(GA_Saatva_Day['ga:country']=="United States") | (GA_Saatva_Day['ga:country']=="Unspecified")]

In [38]:
GA_Saatva_Day_After_ContryUS['ga:sessions'].sum()

952806.0

In [39]:
GA_Saatva_Day_After_ContryUS['ga:region'].unique()

array(['(not set)', 'Alabama', 'Alaska', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [42]:
GA_Saatva_Day_After_Region=GA_Saatva_Day_After_ContryUS[(GA_Saatva_Day_After_ContryUS['ga:region']!="Alaska") & (GA_Saatva_Day_After_ContryUS['ga:region']!="Hawaii")]

In [43]:
GA_Saatva_Day_After_Region['ga:sessions'].sum()

948549.0

In [27]:
GA_Saatva_Day['date']=GA_Saatva_Day['ga:date'].apply(lambda x:datetime.datetime.strptime(x,"%Y%m%d").date())
del GA_Saatva_Day['ga:date']

In [28]:
Saatva_by_Day=pd.merge(Saatva_by_Day,GA_Saatva_Day,on=['brand','date'],how='left')

In [29]:
Saatva_by_Day_Online_Sales=Sales[Sales['brand']=="Saatva"]

In [30]:
Saatva_by_Day_Online=Saatva_by_Day_Online_Sales.groupby(['date','brand','online/phone?'])['product_revenue'].agg(groupby_func)
Saatva_by_Day_Online.columns=Saatva_by_Day_Online.columns.get_level_values(0)
Saatva_by_Day_Online.columns=['sales','transaction']
Saatva_by_Day_Online.reset_index(inplace=True)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [31]:
Saatva_by_Day_Online_Sales=Saatva_by_Day_Online[['date','online/phone?','sales']]
Saatva_by_Day_Online_Sales=Saatva_by_Day_Online_Sales.pivot('date','online/phone?','sales')
Saatva_by_Day_Online_Sales.reset_index(inplace=True)
Saatva_by_Day_Online_Sales.columns=['date','online_sales','phone_sales']

In [32]:
Saatva_by_Day_Online_Trans=Saatva_by_Day_Online[['date','online/phone?','transaction']]
Saatva_by_Day_Online_Trans=Saatva_by_Day_Online_Trans.pivot('date','online/phone?','transaction')
Saatva_by_Day_Online_Trans.reset_index(inplace=True)
Saatva_by_Day_Online_Trans.columns=['date','online_trans','phone_trans']

In [33]:
Saatva_by_Day=pd.merge(Saatva_by_Day,Saatva_by_Day_Online_Sales,on='date',how='left')
Saatva_by_Day=pd.merge(Saatva_by_Day,Saatva_by_Day_Online_Trans,on='date',how='left')

# Saatva_by_Day.to_csv("/home/jian/Projects/Saatva/output/Saatva_GA_Sales_by_Day.csv")
# Merge with Adwords Below

# Google Adwords

In [34]:
# 49+3  'San Francisco Bay Area','Silicon Valley','Unspecified'
Lower49_List=['Alabama','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','District of Columbia',
             'Florida','Georgia','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland',
             'Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire',
             'New Jersey','New Mexico','New York','North Carolina','North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania',
              'Rhode Island','San Francisco Bay Area','Silicon Valley','South Carolina','South Dakota','Tennessee','Texas',
              'Unspecified','Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming']

In [35]:
def clean_data(df):
    if 'Day' in df.columns.tolist():
        df['Day']=df['Day'].apply(lambda x: datetime.datetime.strptime(x,"%b %d, %Y").date())
    if 'Impressions' in df.columns.tolist():
        df['Impressions']=df['Impressions'].str.replace(",","").astype(int)
    if 'Clicks' in df.columns.tolist():
        df['Clicks']=df['Clicks'].str.replace(",","").astype(int)
    if 'Cost' in df.columns.tolist():
        df['Cost']=df['Cost'].str.replace(",","")
        df['Cost']=df['Cost'].str.replace("$","").astype(float)
    df=df[df['Country/Territory'].isin(["United States","Unspecified",np.nan])]
    df=df[df['Region'].isin(Lower49_List)]
    return df

In [36]:
Saatva_GoogleAD_1=pd.read_csv("/home/jian/Projects/Saatva/AdWords/Saatva_20161101_20170430_Day_Type_Country_Region_Campaign.csv",skiprows=2,dtype=str)
Saatva_GoogleAD_2=pd.read_csv("/home/jian/Projects/Saatva/AdWords/Saatva_20170501_20171031_Day_Type_Country_Region_Campaign.csv",skiprows=2,dtype=str)
Saatva_GoogleAD_3=pd.read_csv("/home/jian/Projects/Saatva/AdWords/Saatva_20171101_20180228_Day_Type_Country_Region_Campaign.csv",skiprows=2,dtype=str)

Saatva_GoogleAD=Saatva_GoogleAD_1.append(Saatva_GoogleAD_2).append(Saatva_GoogleAD_3)
Saatva_GoogleAD=clean_data(Saatva_GoogleAD)

x=Saatva_GoogleAD[((Saatva_GoogleAD['Campaign'].str.contains("Leaf")) | (Saatva_GoogleAD['Campaign'].str.contains("Loom")) | (Saatva_GoogleAD['Campaign'].str.contains("Zenhaven")))]
Saatva_GoogleAD=Saatva_GoogleAD[~((Saatva_GoogleAD['Campaign'].str.contains("Leaf")) | (Saatva_GoogleAD['Campaign'].str.contains("Loom")) | (Saatva_GoogleAD['Campaign'].str.contains("Zenhaven")))]


del Saatva_GoogleAD['Campaign']

Saatva_GoogleAD=Saatva_GoogleAD.groupby(['Day','Campaign type','Country/Territory','Region','Currency'])['Impressions','Clicks','Cost'].sum()
Saatva_GoogleAD.reset_index(inplace=True)
Saatva_GoogleAD['Campaign Account']="Saatva"


In [37]:
SaatvaBranding_GoogleAD=pd.read_csv("/home/jian/Projects/Saatva/AdWords/Saatva GSP_20161101_20171031_Day_Type_Country_Region_Campaign.csv",skiprows=2,dtype=str)
SaatvaBranding_GoogleAD=clean_data(SaatvaBranding_GoogleAD)

x=SaatvaBranding_GoogleAD[(SaatvaBranding_GoogleAD['Campaign'].str.contains("Leaf") | SaatvaBranding_GoogleAD['Campaign'].str.contains("Loom") | SaatvaBranding_GoogleAD['Campaign'].str.contains("Zenhaven"))]
SaatvaBranding_GoogleAD=SaatvaBranding_GoogleAD[~(SaatvaBranding_GoogleAD['Campaign'].str.contains("Leaf") | SaatvaBranding_GoogleAD['Campaign'].str.contains("Loom") | SaatvaBranding_GoogleAD['Campaign'].str.contains("Zenhaven"))]


SaatvaBranding_GoogleAD=SaatvaBranding_GoogleAD.groupby(['Day','Campaign type','Country/Territory','Region','Currency'])['Impressions','Clicks','Cost'].sum()
SaatvaBranding_GoogleAD.reset_index(inplace=True)
SaatvaBranding_GoogleAD['Campaign Account']="Saatva Branding"
x['Campaign'].unique()

array([], dtype=object)

In [38]:
SaatvaCompetitor_GoogleAD_1=pd.read_csv("/home/jian/Projects/Saatva/AdWords/Saatva Competitors_20161101_20171031_Day_Type_Country_Region_Campaign.csv",skiprows=2,dtype=str)
SaatvaCompetitor_GoogleAD_2=pd.read_csv("/home/jian/Projects/Saatva/AdWords/Saatva Competitors_20171101_20180228_Day_Type_Country_Region_Campaign.csv",skiprows=2,dtype=str)
SaatvaCompetitor_GoogleAD=SaatvaCompetitor_GoogleAD_1.append(SaatvaCompetitor_GoogleAD_2)
SaatvaCompetitor_GoogleAD=clean_data(SaatvaCompetitor_GoogleAD)

x=SaatvaCompetitor_GoogleAD[(SaatvaCompetitor_GoogleAD['Campaign'].str.contains("Leaf") | SaatvaCompetitor_GoogleAD['Campaign'].str.contains("Loom") | SaatvaCompetitor_GoogleAD['Campaign'].str.contains("Zenhaven"))]
SaatvaCompetitor_GoogleAD=SaatvaCompetitor_GoogleAD[~(SaatvaCompetitor_GoogleAD['Campaign'].str.contains("Leaf") | SaatvaCompetitor_GoogleAD['Campaign'].str.contains("Loom") | SaatvaCompetitor_GoogleAD['Campaign'].str.contains("Zenhaven"))]


SaatvaCompetitor_GoogleAD['Campaign_Brand']=SaatvaCompetitor_GoogleAD['Campaign'].apply(lambda x:x[len(x)-4:len(x)+1])
y=SaatvaCompetitor_GoogleAD[SaatvaCompetitor_GoogleAD['Campaign_Brand']!='[SM]']
SaatvaCompetitor_GoogleAD=SaatvaCompetitor_GoogleAD[SaatvaCompetitor_GoogleAD['Campaign_Brand']=='[SM]']

del SaatvaCompetitor_GoogleAD['Campaign_Brand']
del SaatvaCompetitor_GoogleAD['Campaign']

SaatvaCompetitor_GoogleAD=SaatvaCompetitor_GoogleAD.groupby(['Day','Campaign type','Country/Territory','Region','Currency'])['Impressions','Clicks','Cost'].sum()
SaatvaCompetitor_GoogleAD.reset_index(inplace=True)
SaatvaCompetitor_GoogleAD['Campaign Account']="Saatva Competitors"
x['Campaign'].unique()

array([], dtype=object)

In [39]:
Saatva_GSP_GoogleAD=pd.read_csv("/home/jian/Projects/Saatva/AdWords/Saatva GSP_20161101_20171031_Day_Type_Country_Region_Campaign.csv",skiprows=2,dtype=str)
Saatva_GSP_GoogleAD=clean_data(Saatva_GSP_GoogleAD)

x=Saatva_GSP_GoogleAD[(Saatva_GSP_GoogleAD['Campaign'].str.contains("Leaf") | Saatva_GSP_GoogleAD['Campaign'].str.contains("Loom") | Saatva_GSP_GoogleAD['Campaign'].str.contains("Zenhaven"))]
Saatva_GSP_GoogleAD=Saatva_GSP_GoogleAD[~(Saatva_GSP_GoogleAD['Campaign'].str.contains("Leaf") | Saatva_GSP_GoogleAD['Campaign'].str.contains("Loom") | Saatva_GSP_GoogleAD['Campaign'].str.contains("Zenhaven"))]


Saatva_GSP_GoogleAD=Saatva_GSP_GoogleAD.groupby(['Day','Campaign type','Country/Territory','Region','Currency'])['Impressions','Clicks','Cost'].sum()
Saatva_GSP_GoogleAD.reset_index(inplace=True)
Saatva_GSP_GoogleAD['Campaign type']='Display/Gmail'
Saatva_GSP_GoogleAD['Campaign Account']="Saatva GSP"
x['Campaign'].unique()

array([], dtype=object)

In [40]:
MediaSpend_Saatva=Saatva_GoogleAD.append(SaatvaBranding_GoogleAD).append(SaatvaCompetitor_GoogleAD).append(Saatva_GSP_GoogleAD)

In [41]:
# Because of the low value, "Search Network with Display Select" is replaced by "Search Only"
MediaSpend_Saatva['Campaign type']=MediaSpend_Saatva['Campaign type'].replace("Search Network with Display Select","Search Only")

In [42]:
MediaSpend_Saatva.to_csv("/home/jian/Projects/Saatva/output/Saatva_Google_Ads_with_Account.csv",index=False)

In [43]:
MediaSpend_Saatva_Day=MediaSpend_Saatva.groupby(['Day'])['Impressions','Clicks','Cost'].sum()
MediaSpend_Saatva_Day.reset_index(inplace=True)
MediaSpend_Saatva_Day.columns=['date','AdWords_Impr','AdWords_Clicks','AdWords_Cost']

In [44]:
Saatva_by_Day=pd.merge(Saatva_by_Day,MediaSpend_Saatva_Day,on='date',how='left')
Saatva_by_Day.to_csv("/home/jian/Projects/Saatva/output/Saatva_GA_Ads_Sales_by_Day.csv",index=False)

# Media Spend Wide

In [45]:
print(str(min(MediaSpend_Saatva['Day']))+" to "+str(max(MediaSpend_Saatva['Day'])))
MediaSpend_Saatva_Cost=MediaSpend_Saatva.groupby(['Day','Campaign type'])['Cost'].sum().to_frame()
MediaSpend_Saatva_Cost.reset_index(inplace=True)
MediaSpend_Saatva_Cost=MediaSpend_Saatva_Cost.pivot('Day','Campaign type','Cost')
MediaSpend_Saatva_Cost.reset_index(inplace=True)
MediaSpend_Saatva_Cost=MediaSpend_Saatva_Cost.rename(columns={"Display Only":"Cost:Display","Search Only":"Cost:SEM","Display/Gmail":"Cost:Display/Gmail","Shopping":"Cost:PLA","Video":"Cost:Video",})
MediaSpend_Saatva_Cost=MediaSpend_Saatva_Cost.fillna(0)

2016-11-01 to 2018-02-28


In [46]:
print(str(min(MediaSpend_Saatva['Day']))+" to "+str(max(MediaSpend_Saatva['Day'])))
MediaSpend_Saatva_Impression=MediaSpend_Saatva.groupby(['Day','Campaign type'])['Impressions'].sum().to_frame()
MediaSpend_Saatva_Impression.reset_index(inplace=True)
MediaSpend_Saatva_Impression=MediaSpend_Saatva_Impression.pivot('Day','Campaign type','Impressions')
MediaSpend_Saatva_Impression.reset_index(inplace=True)
MediaSpend_Saatva_Impression=MediaSpend_Saatva_Impression.rename(columns={"Display Only":"Impressions:Display","Search Only":"Impressions:SEM","Display/Gmail":"Impressions:Display/Gmail","Shopping":"Impressions:PLA","Video":"Impressions:Video",})
MediaSpend_Saatva_Impression=MediaSpend_Saatva_Impression.fillna(0)

2016-11-01 to 2018-02-28


In [47]:
print(str(min(MediaSpend_Saatva['Day']))+" to "+str(max(MediaSpend_Saatva['Day'])))
MediaSpend_Saatva_Click=MediaSpend_Saatva.groupby(['Day','Campaign type'])['Clicks'].sum().to_frame()
MediaSpend_Saatva_Click.reset_index(inplace=True)
MediaSpend_Saatva_Click=MediaSpend_Saatva_Click.pivot('Day','Campaign type','Clicks')
MediaSpend_Saatva_Click.reset_index(inplace=True)
MediaSpend_Saatva_Click=MediaSpend_Saatva_Click.rename(columns={"Display Only":"Clicks:Display","Search Only":"Clicks:SEM","Display/Gmail":"Clicks:Display/Gmail","Shopping":"Clicks:PLA","Video":"Clicks:Video",})
MediaSpend_Saatva_Click=MediaSpend_Saatva_Click.fillna(0)

2016-11-01 to 2018-02-28


In [48]:
MediaSpend=pd.merge(MediaSpend_Saatva_Cost,MediaSpend_Saatva_Impression,on='Day',how='outer')
MediaSpend=pd.merge(MediaSpend,MediaSpend_Saatva_Click,on='Day',how='outer')

In [49]:
MediaSpend.to_csv("/home/jian/Projects/Saatva/output/Saatva_GoogleAdWords_by_Day.csv",index=False)

In [50]:
MediaSpend=MediaSpend.rename(columns={"Day":"date"})

In [51]:
Saatva_by_Day_Media=pd.merge(Saatva_by_Day,MediaSpend,on="date",how="left")

In [52]:
Saatva_by_Day_Media.to_csv("/home/jian/Projects/Saatva/output/Saatva_GoogleAdWords_by_Day_Media.csv",index=False)

In [53]:
zip_data=pd.read_excel("/home/jian/Projects/Saatva/Tableau/Zip_Tableau.xlsx",dtype=str)

In [54]:
all_zip=pd.read_excel("/home/jian/Projects/Saatva/Tableau/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)

In [55]:
all_zip=all_zip[['CODE','NAME']]
all_zip.columns=['zip5','DMA_Name']
all_zip=all_zip.drop_duplicates()
all_zip.shape

(43102, 2)

In [56]:
all_zip_data=pd.merge(all_zip,zip_data,on="zip5",how='outer')

In [57]:
all_zip_data.to_excel("/home/jian/Projects/Saatva/Tableau/All_Zip_Tableau.xlsx",index=False)